# Getting and filtering data for each game for seasons from 2011-2022 (12 seasons)

In [48]:
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoretraditionalv2
import pandas as pd

COUNTER =  1
DIRECTION = "ASC"
LEAGUE = "00"
PLAYER_OR_TEAM = "T"
SEASON_TYPE = "Regular Season"
SORTER = "DATE"

def fetch_season_game_logs(season):
    # Create LeagueGameLog object instance for the season
    gamelog = leaguegamelog.LeagueGameLog(
    COUNTER, DIRECTION, LEAGUE, PLAYER_OR_TEAM, season, SEASON_TYPE, SORTER
)
    
    # Execute request and fetch data
    data = gamelog.get_data_frames()[0] 
    
    # Filter columns and add Home/Away column
    columns_to_keep = ["TEAM_ID","TEAM_NAME","GAME_ID","MATCHUP","WL", "GAME_DATE"]
    filtered_data = data[columns_to_keep].copy()
    filtered_data['HOME/AWAY'] = filtered_data['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    
    return filtered_data

def fetch_multiple_seasons(start_year, end_year):
    all_seasons_data = []  # List to store data for all seasons
    
    # Loop through each season from start_year to end_year (inclusive)
    for year in range(start_year, end_year + 1):
        season = f"{year}-{str(year + 1)[-2:]}"  # Format season string (e.g., '2010-11')
        season_data = fetch_season_game_logs(season)
        all_seasons_data.append(season_data)
    
    # Concatenate all seasons' data into a single DataFrame
    combined_data = pd.concat(all_seasons_data, ignore_index=True)
    combined_data = combined_data.sort_values(by='GAME_ID', ascending=True)
    combined_data = combined_data.reset_index(drop=True)

    team_2_id = []
    team_2_name = []

    # Add new columns for team IDs and names from adjacent rows
    for i in range(len(combined_data)):
        if i % 2 == 0:  # Every even row
            team_2_id.append(combined_data.loc[i+1, 'TEAM_ID'])
            team_2_name.append(combined_data.loc[i+1, 'TEAM_NAME'])
        else:  # Every odd row
            team_2_id.append(combined_data.loc[i-1, 'TEAM_ID'])
            team_2_name.append(combined_data.loc[i-1, 'TEAM_NAME'])
    
    # Add new columns to the DataFrame
    combined_data['TEAM_2_ID'] = team_2_id
    combined_data['TEAM_2_NAME'] = team_2_name
    
    columns_order = ["GAME_ID", "GAME_DATE", "MATCHUP", "TEAM_NAME", "TEAM_ID", "TEAM_2_NAME", "TEAM_2_ID", "WL", "HOME/AWAY"]
    combined_data = combined_data[columns_order]
    
    return combined_data

print(fetch_multiple_seasons(2023,2023))

         GAME_ID   GAME_DATE      MATCHUP             TEAM_NAME     TEAM_ID  \
0     0022300001  2023-11-03    CLE @ IND   Cleveland Cavaliers  1610612739   
1     0022300001  2023-11-03  IND vs. CLE        Indiana Pacers  1610612754   
2     0022300002  2023-11-03  MIL vs. NYK       Milwaukee Bucks  1610612749   
3     0022300002  2023-11-03    NYK @ MIL       New York Knicks  1610612752   
4     0022300003  2023-11-03  MIA vs. WAS            Miami Heat  1610612748   
...          ...         ...          ...                   ...         ...   
2455  0022301228  2023-12-08    SAC @ PHX      Sacramento Kings  1610612758   
2456  0022301229  2023-12-07  MIL vs. IND       Milwaukee Bucks  1610612749   
2457  0022301229  2023-12-07    IND @ MIL        Indiana Pacers  1610612754   
2458  0022301230  2023-12-07    NOP @ LAL  New Orleans Pelicans  1610612740   
2459  0022301230  2023-12-07  LAL vs. NOP    Los Angeles Lakers  1610612747   

               TEAM_2_NAME   TEAM_2_ID WL  HOME/AWA

# Aquiring Season Stats for each team 

In [49]:
from api_helpers.team_stats_helpers import load_dataframe

attribute_list = ["PTS","FGM","FGA","FG_PCT","FG3M","FG3A","FG3_PCT","FTM","FTA","FT_PCT","OREB","DREB","AST"]
nba_dataframe = load_dataframe(attribute_list)
nba_dataframe = nba_dataframe.drop(nba_dataframe[nba_dataframe["FGM"] == 0].index)
nba_dataframe.sort_values(by='YEAR')

nba_dataframe['YEAR'] = nba_dataframe['YEAR'].str.split('-').str[0]
nba_dataframe['YEAR'] = pd.to_numeric(nba_dataframe['YEAR'])
nba_dataframe.sort_values(by="YEAR",inplace=True)

nba_dataframe["NBA_FINALS_APPEARANCE"].fillna(0.0, inplace=True)
nba_dataframe["NBA_FINALS_APPEARANCE"].replace("FINALS APPEARANCE", 0.0, inplace=True)
nba_dataframe["NBA_FINALS_APPEARANCE"].replace("LEAGUE CHAMPION", 2, inplace=True)

nba_dataframe = nba_dataframe[nba_dataframe['YEAR'] >= 2023]
season_stat_dataframe = nba_dataframe.reset_index(drop=True)

print(nba_dataframe)



         TEAM_ID  YEAR  NBA_FINALS_APPEARANCE   FGM   FGA  FG_PCT  FG3M  FG3A  \
74    1610612737  2023                    0.0  3529  7584   0.465  1125  3092   
152   1610612738  2023                    2.0  3601  7396   0.487  1351  3482   
1374  1610612761  2023                    0.0  3466  7356   0.471   942  2712   
1453  1610612763  2023                    0.0  3145  7230   0.435  1071  3097   
1424  1610612762  2023                    0.0  3443  7371   0.467  1060  2993   
1240  1610612758  2023                    0.0  3553  7455   0.477  1178  3219   
1288  1610612759  2023                    0.0  3438  7436   0.462  1036  2983   
1345  1610612760  2023                    0.0  3653  7324   0.499  1090  2805   
643   1610612747  2023                    0.0  3580  7177   0.499   969  2572   
1164  1610612757  2023                    0.0  3227  7356   0.439   939  2723   
679   1610612748  2023                    0.0  3266  7022   0.465  1022  2765   
567   1610612746  2023      

# Combining Dataframes
For each game, listing the stats of both teams for comparison

In [50]:
game_data = fetch_multiple_seasons(2023,2023)
season_team_data = season_stat_dataframe


team_1_pts = []
team_2_pts = []

 # Add new columns for team IDs and names from adjacent rows
for i in range(len(game_data)):
    team_1_pts.append(season_team_data.loc[season_team_data[season_team_data['TEAM_ID'] == game_data.loc[i, 'TEAM_ID']].index[0], 'PTS'])
    team_2_pts.append(season_team_data.loc[season_team_data[season_team_data['TEAM_ID'] == game_data.loc[i, 'TEAM_2_ID']].index[0], 'PTS'])

game_data['TEAM_1_PTS'] = team_1_pts
game_data['TEAM_2_PTS'] = team_2_pts

print(game_data)





         GAME_ID   GAME_DATE      MATCHUP             TEAM_NAME     TEAM_ID  \
0     0022300001  2023-11-03    CLE @ IND   Cleveland Cavaliers  1610612739   
1     0022300001  2023-11-03  IND vs. CLE        Indiana Pacers  1610612754   
2     0022300002  2023-11-03  MIL vs. NYK       Milwaukee Bucks  1610612749   
3     0022300002  2023-11-03    NYK @ MIL       New York Knicks  1610612752   
4     0022300003  2023-11-03  MIA vs. WAS            Miami Heat  1610612748   
...          ...         ...          ...                   ...         ...   
2455  0022301228  2023-12-08    SAC @ PHX      Sacramento Kings  1610612758   
2456  0022301229  2023-12-07  MIL vs. IND       Milwaukee Bucks  1610612749   
2457  0022301229  2023-12-07    IND @ MIL        Indiana Pacers  1610612754   
2458  0022301230  2023-12-07    NOP @ LAL  New Orleans Pelicans  1610612740   
2459  0022301230  2023-12-07  LAL vs. NOP    Los Angeles Lakers  1610612747   

               TEAM_2_NAME   TEAM_2_ID WL  HOME/AWA